<a href="https://colab.research.google.com/github/Rattapon-Insa/time-series/blob/main/LSTM_on_NIFTY_50_Stock_Market_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rohanrao/nifty50-stock-market-data

In [ ]:
! unzip nifty50-stock-market-data.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('HCLTECH.csv')

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'])
df.dropna()
df.set_index(df['Date'], inplace = True)
df = df[['Prev Close']]['2013-01-01':'2013-12-02']

In [ ]:
# ref index

train_size = 210
train_in = df[:train_size]
test_in = df[train_size:]

In [ ]:
max_size = df.values.shape[0]
test_slot = max_size-train_size

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(train_in.values, start_p =1,
                      start_q = 0,
                      test = 'adf', 
                      max_p =3,
                      max_q = 3,
                      m = 1,
                      d = None,
                      seasonal = False,
                      start_P = 0,
                      D = 0,
                      trace = True,
                      error_action = 'ignore',
                      suppress_warnings = True,
                      stepwise = True) 

print(model.summary())

In [ ]:
model.plot_diagnostics(figsize = (7,5))
plt.show()

In [ ]:
# forecast
fc, conf = model.predict(test_slot, return_conf_int = True)

# Make a series
fc_series = pd.Series(fc, index = test_in.index)
lower_series = pd.Series(conf[:,0], index = test_in.index)
upper_series = pd.Series(conf[:, 1], index = test_in.index)

In [ ]:
# plot

plt.figure(figsize = (12,5), dpi = 100)
plt.plot(train_in, label = 'Training')
plt.plot(test_in, label = 'Actual')
plt.plot(fc_series, label = 'forecast')
plt.fill_between(lower_series.index, lower_series, upper_series
                 , color = 'k', alpha = 0.15)

plt.title('Forecast vs Actual')
plt.legend()